In [1]:
%load_ext autotime

import os
import sys
import math
import copy
import random
import warnings
import pandas as pd
import numpy as np

random.seed(1)
warnings.filterwarnings('ignore')
from tqdm import tqdm, trange

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '4, 5, 6, 7'

time: 477 µs


In [3]:
import logging

log_format = '%(levelname)s %(asctime)s - %(message)s'
logging.basicConfig(filename = '../logs/original_fsl.logs',
                    level = logging.INFO,
                    format = log_format,
                    filemode = 'w')
logger = logging.getLogger()

time: 2.26 ms


In [4]:
train_data = pd.read_csv('../data/train_data.csv')
print (train_data.shape)
print (train_data.columns)

(368945, 19)
Index(['subject_id', 'image_path', 'image_name', 'study_id', 'split',
       'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
       'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity',
       'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia',
       'Pneumothorax', 'Support Devices'],
      dtype='object')
time: 2.13 s


In [5]:
train_data.head(5)

,subject_id,image_path,image_name,study_id,split,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,s52769454,../../DataCenter/MIMIC-CXR/files/p18/p18190098...,8bf006fa-7169cd83-c30e7055-b109468a-2223477d,52769454,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,s50754262,../../DataCenter/MIMIC-CXR/files/p18/p18190098...,48fbe534-50750d68-5afd36c2-e7aaf316-a31fcb66,50754262,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,s51845898,../../DataCenter/MIMIC-CXR/files/p18/p18190098...,fc2e907b-c19b5434-cc3d4205-8d66ab01-a0dcc274,51845898,train,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,s51845898,../../DataCenter/MIMIC-CXR/files/p18/p18190098...,e921e838-0c4b1f64-0572971d-b67b4730-4182681f,51845898,train,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,s52846638,../../DataCenter/MIMIC-CXR/files/p18/p18190098...,c458d43d-f4a7eb7d-a51b95a8-f4aa2c82-52d243ff,52846638,train,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


time: 44.9 ms


In [6]:
import cv2
import csv
import time
import random
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optin
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as tfunc
import torch.multiprocessing as mp

from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.parallel import DistributedDataParallel as DDP
from PIL import Image

from sklearn.metrics import roc_auc_score, confusion_matrix
from tqdm import tqdm, trange
import sklearn.metrics as metrics

use_gpu = torch.cuda.is_available()

time: 933 ms


In [7]:
sample_train_data = train_data[0:10000]
sample_train_data.to_csv('../data/sample_train_data.csv', index = False)

time: 168 ms


In [8]:
pathFileSampleTrain = '../data/sample_train_data.csv'
pathFileTrain = '../data/train_data.csv'
pathFileValid = '../data/val_data.csv'

nnIsTrained = False
nnClassCount = 14

trBatchSize = 64
trMaxEpoch = 3

imgtransResize = (320, 320)
imgtransCrop = 224

class_names = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 
               'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 
               'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

time: 1.18 ms


In [9]:
class CheXpertDataSet(Dataset):
    def __init__(self, image_list_file, transform=None, policy="ones"):
        """
        image_list_file: path to the file containing images with corresponding labels.
        transform: optional transform to be applied on a sample.
        policy: name the policy with regard to the uncertain labels
        """
        image_names = []
        labels = []

        with open(image_list_file, "r") as f:
            csvReader = csv.reader(f)
            next(csvReader, None)
            k=0
            for line in csvReader:
                k+=1
                image_name= line[1]
                label = line[5:]
                
                for i in range(14):
                    label[i] = float(label[i])
                
                image_names.append(image_name)
                labels.append(label)

        self.image_names = image_names
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        """Take the index of item and returns the image and its labels"""
        
        image_name = self.image_names[index]
        image = Image.open(image_name).convert('RGB')
        label = self.labels[index]
        if self.transform is not None:
            image = self.transform(image)
        return (image, torch.FloatTensor(label))

    def __len__(self):
        return (len(self.image_names))

time: 3.48 ms


In [10]:
#TRANSFORM DATA

normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transformList = []
transformList.append(transforms.RandomResizedCrop(imgtransCrop))
transformList.append(transforms.RandomHorizontalFlip())
transformList.append(transforms.ToTensor())
transformList.append(normalize)      
transformSequence=transforms.Compose(transformList)

time: 1.18 ms


In [11]:
#LOAD DATASET

datasetSampleTrain = CheXpertDataSet(pathFileSampleTrain, transformSequence, policy = 'ones')
datasetTrain = CheXpertDataSet(pathFileTrain ,transformSequence, policy = 'ones')
datasetValid = CheXpertDataSet(pathFileValid, transformSequence)

print ('Size of Simple Train Dataset - {}'.format(len(datasetSampleTrain)))
print ('Size of Train Dataset - {}'.format(len(datasetTrain)))

dataLoaderSampleTrain = DataLoader(dataset=datasetSampleTrain, batch_size=trBatchSize, shuffle=True, num_workers=0, pin_memory = True)
dataLoaderTrain = DataLoader(dataset=datasetTrain, batch_size=trBatchSize, shuffle=True,  num_workers=0, pin_memory = True)
dataLoaderVal = DataLoader(dataset=datasetValid, batch_size=trBatchSize, shuffle=False, num_workers=0, pin_memory = True)

Size of Simple Train Dataset - 10000
Size of Train Dataset - 368945
time: 2.27 s


In [12]:
class CheXpertTrainer():

    def train (model, dataLoaderTrain, dataLoaderVal, nnClassCount, trMaxEpoch, launchTimestamp, checkpoint):
        
        #SETTINGS: OPTIMIZER & SCHEDULER
        optimizer = optim.Adam (model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-5)
                
        #SETTINGS: LOSS
        loss = torch.nn.BCELoss(size_average = True)
        
        #LOAD CHECKPOINT 
        if checkpoint != None and use_gpu:
            modelCheckpoint = torch.load(checkpoint)
            model.load_state_dict(modelCheckpoint['state_dict'])
            optimizer.load_state_dict(modelCheckpoint['optimizer'])
da
        
        #TRAIN THE NETWORK
        lossMIN = 100000
        
        for epochID in range(0, trMaxEpoch):
            
            timestampTime = time.strftime("%H%M%S")
            timestampDate = time.strftime("%d%m%Y")
            timestampSTART = timestampDate + '-' + timestampTime
            
            batchs, losst, losse = CheXpertTrainer.epochTrain(model, dataLoaderTrain, optimizer, trMaxEpoch, nnClassCount, loss)
            lossVal = CheXpertTrainer.epochVal(model, dataLoaderVal, optimizer, trMaxEpoch, nnClassCount, loss)


            timestampTime = time.strftime("%H%M%S")
            timestampDate = time.strftime("%d%m%Y")
            timestampEND = timestampDate + '-' + timestampTime
            
            if lossVal < lossMIN:
                lossMIN = lossVal    
                torch.save({'epoch': epochID + 1, 'state_dict': model.state_dict(), 'best_loss': lossMIN, 'optimizer' : optimizer.state_dict()}, 'm-epoch'+str(epochID)+'-' + launchTimestamp + '.pth.tar')
                print ('Epoch [' + str(epochID + 1) + '] [save] [' + timestampEND + '] loss= ' + str(lossVal))
            else:
                print ('Epoch [' + str(epochID + 1) + '] [----] [' + timestampEND + '] loss= ' + str(lossVal))
        
        return (batchs, losst, losse)
    #-------------------------------------------------------------------------------- 
       
    def epochTrain(model, dataLoader, optimizer, epochMax, classCount, loss):
        
        batch = []
        losstrain = []
        losseval = []
        
        model.train()

        for batchID, (varInput, target) in enumerate(dataLoaderTrain):
            
            varTarget = target.cuda(non_blocking = True)
            
            #varTarget = target.cuda()         
            
            print (varInput.shape)
            varOutput = model(varInput)
            lossvalue = loss(varOutput, varTarget)
                       
            optimizer.zero_grad()
            lossvalue.backward()
            optimizer.step()
            
            l = lossvalue.item()
            losstrain.append(l)
            
            if batchID%35==0:
                logger.info('Batches Computed - {}'.format(batchID//35))
                print(batchID//35, "% batches computed")
                #Fill three arrays to see the evolution of the loss


                batch.append(batchID)
                
                le = CheXpertTrainer.epochVal(model, dataLoaderVal, optimizer, trMaxEpoch, nnClassCount, loss).item()
                losseval.append(le)
                
                logger.info('Batch ID - {}'.format(batchID))
                logger.info('Training Loss - {}'.format(l))
                logger.info('Validation Loss - {}'.format(le))
                
                print('Batch ID - {}'.format(batchID))
                print('Training Loss - {}'.format(l))
                print('Validation Loss - {}'.format(le))
                
        return (batch, losstrain, losseval)
    
    #-------------------------------------------------------------------------------- 
    
    def epochVal(model, dataLoader, optimizer, epochMax, classCount, loss):
        
        model.eval()
        
        lossVal = 0
        lossValNorm = 0

        with torch.no_grad():
            for i, (varInput, target) in enumerate(dataLoaderVal):
                
                target = target.cuda(non_blocking = True)
                varOutput = model(varInput)
                
                losstensor = loss(varOutput, target)
                lossVal += losstensor
                lossValNorm += 1
                
        outLoss = lossVal / lossValNorm
        return (outLoss)
    
    
    #--------------------------------------------------------------------------------     
     
    #---- Computes area under ROC curve 
    #---- dataGT - ground truth data
    #---- dataPRED - predicted data
    #---- classCount - number of classes
    
    def computeAUROC (dataGT, dataPRED, classCount):
        
        outAUROC = []
        
        datanpGT = dataGT.cpu().numpy()
        datanpPRED = dataPRED.cpu().numpy()
        
        for i in range(classCount):
            try:
                outAUROC.append(roc_auc_score(datanpGT[:, i], datanpPRED[:, i]))
            except ValueError:
                pass
        return (outAUROC)
        
        
    #-------------------------------------------------------------------------------- 
    
    
    def test(model, dataLoaderTest, nnClassCount, checkpoint, class_names):   
        
        cudnn.benchmark = True
        
        if checkpoint != None and use_gpu:
            modelCheckpoint = torch.load(checkpoint)
            model.load_state_dict(modelCheckpoint['state_dict'])

        if use_gpu:
            outGT = torch.FloatTensor().cuda()
            outPRED = torch.FloatTensor().cuda()
        else:
            outGT = torch.FloatTensor()
            outPRED = torch.FloatTensor()
       
        model.eval()
        
        with torch.no_grad():
            for i, (input, target) in enumerate(dataLoaderTest):

                target = target.cuda()
                outGT = torch.cat((outGT, target), 0).cuda()
                
                bs, c, h, w = input.size()
                varInput = input.view(-1, c, h, w)
            
                out = model(varInput)
                outPRED = torch.cat((outPRED, out), 0)
        aurocIndividual = CheXpertTrainer.computeAUROC(outGT, outPRED, nnClassCount)
        aurocMean = np.array(aurocIndividual).mean()
        
        logger.info('AUROC mean ', aurocMean)
        print ('AUROC mean ', aurocMean)
        
        for i in range (0, len(aurocIndividual)):
            print (class_names[i], ' ', aurocIndividual[i])
        
        return (outGT, outPRED)

time: 7.19 ms


In [13]:
class DenseNet121(nn.Module):
    """Model modified.
    The architecture of our model is the same as standard DenseNet121
    except the classifier layer which has an additional sigmoid function.
    """
    def __init__(self, out_size):
        super(DenseNet121, self).__init__()
        self.densenet121 = torchvision.models.densenet121(pretrained=True)
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_ftrs, out_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.densenet121(x)
        return (x)

time: 895 µs


In [14]:
# initialize and load the model

model = DenseNet121(nnClassCount).cuda()
model = torch.nn.DataParallel(model).cuda()
logger.info('Initial model is loaded on to GPU.')

time: 2.5 s


In [15]:
pretrained_model = torch.load('../model/model_ones_3epoch_densenet.tar')
model_state_dict = pretrained_model['state_dict']
torch.save(model_state_dict, '../model/model_state_dict.pth')

time: 159 ms


In [16]:
model.load_state_dict(torch.load('../model/model_state_dict.pth'))
model.eval()

DataParallel(
  (module): DenseNet121(
    (densenet121): DenseNet(
      (features): Sequential(
        (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu0): ReLU(inplace=True)
        (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (denseblock1): _DenseBlock(
          (denselayer1): _DenseLayer(
            (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace=True)
            (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace=True)
            (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          )
          (denselayer2): _Dens

time: 100 ms


In [17]:
outGT, outPRED = CheXpertTrainer.test(model = model, dataLoaderTest = dataLoaderSampleTrain, nnClassCount = nnClassCount, class_names = class_names, checkpoint = None)

AUROC mean  0.527075225168402
No Finding   0.2608430020766045
Enlarged Cardiomediastinum   0.48847670167915547
Cardiomegaly   0.5818568200043741
Lung Opacity   0.7869942444553917
Lung Lesion   0.4057614511961206
Edema   0.49344914364772674
Consolidation   0.5791579601316831
Pneumonia   0.6919774343070435
Atelectasis   0.22803036526434659
Pneumothorax   0.4570719185780653
Pleural Effusion   0.606292409468902
Pleural Other   0.505209144749428
Fracture   0.4341911618062391
Support Devices   0.8597413949925465
time: 9min 36s


--- Logging error ---
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.7/logging/__init__.py", line 1034, in emit
    msg = self.format(record)
  File "/opt/tljh/user/lib/python3.7/logging/__init__.py", line 880, in format
    return fmt.format(record)
  File "/opt/tljh/user/lib/python3.7/logging/__init__.py", line 619, in format
    record.message = record.getMessage()
  File "/opt/tljh/user/lib/python3.7/logging/__init__.py", line 380, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/tljh/user/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/tljh/user/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/tljh/user/lib/python3.7/site-packages/traitlets/config/application.py", line 

In [18]:
outPRED = outPRED.cpu().detach().numpy()
outPRED = np.array(outPRED)

sample_train_data['pred_No Finding'] = np.round(outPRED[:,0])
sample_train_data['pred_Enlarged Cardiomediastinum'] = np.round(outPRED[:,1])
sample_train_data['pred_Cardiomegaly'] = np.round(outPRED[:,2])
sample_train_data['pred_Lung Opacity'] = np.round(outPRED[:,3])
sample_train_data['pred_Lung Lesion'] = np.round(outPRED[:,4])
sample_train_data['pred_Edema'] = np.round(outPRED[:,5])
sample_train_data['pred_Consolidation'] = np.round(outPRED[:,6])
sample_train_data['pred_Pneumonia'] = np.round(outPRED[:,7])
sample_train_data['pred_Atelectasis'] = np.round(outPRED[:,8])
sample_train_data['pred_Pneumothorax'] = np.round(outPRED[:,9])
sample_train_data['pred_Pleural Effusion'] = np.round(outPRED[:,10])
sample_train_data['pred_Pleural Other'] = np.round(outPRED[:,11])
sample_train_data['pred_Fracture'] = np.round(outPRED[:,12])
sample_train_data['pred_Support Devices'] = np.round(outPRED[:,13])

time: 18 ms


In [19]:
len(sample_train_data)

10000

time: 2.18 ms


In [20]:
'pred_No Finding' in sample_train_data.columns

True

time: 2.23 ms


## Todo List

- Train few shot learning model on every class with 100 images.
    - Incrementally build the few shot learning model on all the 14 conditions.
- Validate the results on the all images (except the 10 images).
- Compare the results with the prior results.

In [21]:
args = {
    'train_size' : 100,
    'test_size' : 100,
    'out_size' : 128, # should be less than 512
    'loss_margin' : 0.2,
    'loss_p' : 2,
    'batch_size' : 4 # do not change this
}

time: 753 µs


In [22]:
class DenseNet121ToDense(nn.Module):
    '''
    The architecture of the densenet121 is copied and the final classifier layer is removed.
    In the place of final classifier layer a dense layer is attached with PReLU activation.
    '''
    def __init__(self, out_size):
        super(DenseNet121ToDense, self).__init__()
        self.densenet121todense = model.module.densenet121
        num_ftrs = self.densenet121todense.classifier[0].in_features
        self.densenet121todense.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.PReLU(),
            nn.Linear(512, out_size)
        )
        
    def forward(self, x_1, x_2, x_3):
        x_1 = self.densenet121todense(x_1)
        x_2 = self.densenet121todense(x_2)
        x_3 = self.densenet121todense(x_3)
        return (x_1, x_2, x_3)
    
ref_model = DenseNet121ToDense(args['out_size']).cuda()
ref_model = torch.nn.DataParallel(ref_model).cuda()
logger.info('FSL model is loaded on to GPU.')

time: 24.8 ms


In [23]:
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
import torch.optim as optim
from sklearn.metrics import roc_auc_score

def make_train_triplets(pathology, sample_train_data, n):
    '''
    Returns 'n' triplet images for training data for a given pathology.
    First Image - Any random image (with atleast 40 images which has the pathology)
    Second Image - An image which is positive for the pathology.
    Third Image - An image which is negative for the pathology.
    '''
    
    actual_path = pathology
    pred_path = 'pred_'+pathology
    false_negative = sample_train_data[(sample_train_data[actual_path] == 1.0) & (sample_train_data[pred_path] == 0.0)]
    false_positive = sample_train_data[(sample_train_data[actual_path] == 0.0) & (sample_train_data[pred_path] == 1.0)]
    print ('False Negatives - {} False Positives - {}'.format(len(false_negative), len(false_positive)))

    positive = sample_train_data[sample_train_data[actual_path] == 1.0]
    negative = sample_train_data[sample_train_data[actual_path] == 0.0]
    
    images = []
    checking_label = []
    
    for i in range(n):
        if (len(false_negative.index) > 0 and len(false_positive.index) > 0):
            choose = random.choice(['a','b'])    
            if (choose == 'a'):
                first_image_index = random.choice(false_negative.index)
                checking_label.append(-1)
            elif (choose == 'b'):
                first_image_index = random.choice(false_positive.index)
                checking_label.append(1)
        else:
            if (len(false_negative.index) > 0):
                first_image_index = random.choice(false_negative.index)
                checking_label.append(-1)
            elif (len(false_positive.index) > 0):
                first_image_index = random.choice(false_positive.index)
                checking_label.append(1)
            
        second_image_index = random.choice(positive.index)
        third_image_index = random.choice(negative.index)

        # Convert the indices to images
        first_image = sample_train_data['image_path'][first_image_index]
        first_image = Image.open(first_image).convert('RGB')
        first_image = transformSequence(first_image)
        
        second_image = sample_train_data['image_path'][second_image_index]
        second_image = Image.open(second_image).convert('RGB')
        second_image = transformSequence(second_image)
        
        third_image = sample_train_data['image_path'][third_image_index]
        third_image = Image.open(third_image).convert('RGB')
        third_image = transformSequence(third_image)
        
        images += [[first_image, second_image, third_image]]
        
    return (images, checking_label)

time: 5.04 ms


In [24]:
def make_test_triplets(pathology, sample_train_data):
    '''
    - Returns three images (so the name triplets)
    - First Image : An Inference failed image. (Inference failures can appear in the form of false negatives and 
    false postives)
    - Second Image: An image which is positive for the pathology.
    - Third Image: An image which is negative for the pathology.
    '''
    
    actual_path = pathology
    pred_path = 'pred_'+pathology
    false_negative = sample_train_data[(sample_train_data[actual_path] == 1.0) & (sample_train_data[pred_path] == 0.0)]
    false_positive = sample_train_data[(sample_train_data[actual_path] == 0.0) & (sample_train_data[pred_path] == 1.0)]
    print ('False Negatives - {} False Positives - {}'.format(len(false_negative), len(false_positive)))

    positive = sample_train_data[sample_train_data[actual_path] == 1.0]
    negative = sample_train_data[sample_train_data[actual_path] == 0.0]
    
    images = []
    checking_label = []
    
    false_inference_index = false_negative.index.union(false_positive.index)
    for index in false_inference_index:
        if index in false_negative.index:
            checking_label.append(-1)
        elif index in false_positive.index:
            checking_label.append(1)
            
        first_image_index  = index    
        second_image_index = random.choice(positive.index)
        third_image_index = random.choice(negative.index)

        # Convert the indices to images
        first_image = sample_train_data['image_path'][first_image_index]
        first_image = Image.open(first_image).convert('RGB')
        first_image = transformSequence(first_image)

        second_image = sample_train_data['image_path'][second_image_index]
        second_image = Image.open(second_image).convert('RGB')
        second_image = transformSequence(second_image)

        third_image = sample_train_data['image_path'][third_image_index]
        third_image = Image.open(third_image).convert('RGB')
        third_image = transformSequence(third_image)

        images += [[first_image, second_image, third_image]]
        
    return (images, checking_label, false_inference_index)

time: 3.56 ms


In [25]:
def ref_train_image_triplets(images, checking_label):
    first_images = [i[0] for i in images]
    second_images = [i[1] for i in images]
    third_images = [i[2] for i in images]

    first_images = torch.stack(first_images)
    second_images = torch.stack(second_images)
    third_images = torch.stack(third_images)
    
    target = torch.Tensor(checking_label).int()    
    return (first_images, second_images, third_images, target)


def ref_test_image_triplets(test_images, test_checking_label):
    test_first_images = [i[0] for i in test_images]
    test_second_images = [i[1] for i in test_images]
    test_third_images = [i[2] for i in test_images]
    
    test_first_images = torch.stack(test_first_images)
    test_second_images = torch.stack(test_second_images)
    test_third_images = torch.stack(test_third_images)
    
    test_target = torch.Tensor(test_checking_label).int()
    return (test_first_images, test_second_images, test_third_images, test_target)

time: 2.51 ms


In [26]:
def triplet_distance(anchor, positive, negative):
    '''
    The function calculates the distance between anchor, positive and anchor, negative images. The difference 
    between the pos_distance, neg_distance was calculated and tuned according to the checking_label.
    '''
    pos_distance = F.pairwise_distance(anchor, positive, 2)
    neg_distance = F.pairwise_distance(anchor, negative, 2)
    return (pos_distance, neg_distance)

def predictive_value(conf_matrix):
    '''
    This function returns the positive predictive value and negative predictive value of a classifier.
    Parameters:
    
    1. conf_matrix : A Confusion matrix from the result of a classifier.
    '''
    ppv = (conf_matrix[0][0]/(conf_matrix[0][0] + conf_matrix[0][1]))*100
    npv = (conf_matrix[1][1]/(conf_matrix[1][0] + conf_matrix[1][1]))*100
    return (ppv, npv)

def train_and_eval(ref_model, train_dataloader, test_dataloader):
    for epoch in trange(5):
    
        tr_loss = 0
        nb_tr_steps = 0
        ref_model.train()
        for step, batch in enumerate(train_dataloader):
            anchor, positive, negative, target = batch[0], batch[1], batch[2], batch[3]
            anchor, positive, negative, target = Variable(anchor), Variable(positive), Variable(negative), Variable(target)

            bs, c, h, w = anchor.size()
            anchor_input = anchor.view(-1, c, h, w)
            bs, c, h, w = positive.size()
            positive_input = positive.view(-1, c, h, w)
            bs, c, h, w = negative.size()
            negative_input = negative.view(-1, c, h, w)

            E1, E2, E3 = ref_model(anchor_input, positive_input, negative_input)
            dist_E1_E2, dist_E1_E3 = triplet_distance(E1, E2, E3)

            target = target.cuda()
            loss = criterion(dist_E1_E2, dist_E1_E3, target)
            tr_loss += loss
            nb_tr_steps += 1

            torch.autograd.set_detect_anomaly(True)
            optimizer.zero_grad()
            loss.backward(retain_graph = True)
            optimizer.step()

        print("Train loss: {}".format(tr_loss/nb_tr_steps))

        pred_list = []
        with torch.no_grad():
            ref_model.eval()
            for step, batch in enumerate(test_dataloader):
                anchor, positive, negative, target = batch[0], batch[1], batch[2], batch[3]
                anchor, positive, negative, target = Variable(anchor), Variable(positive), Variable(negative), Variable(target)

                bs, c, h, w = anchor.size()
                anchor_input = anchor.view(-1, c, h, w)
                bs, c, h, w = positive.size()
                positive_input = positive.view(-1, c, h, w)
                bs, c, h, w = negative.size()
                negative_input = negative.view(-1, c, h, w)

                E1, E2, E3 = ref_model(anchor_input, positive_input, negative_input)
                dist_E1_E2, dist_E1_E3 = triplet_distance(E1, E2, E3)

                for i in range(len(dist_E1_E2)):
                    if (dist_E1_E2[i] > dist_E1_E3[i]):
                        pred_list.append(1)
                    else:
                        pred_list.append(0)

        mod_test_target = []
        for i in test_target:
            if i == -1:
                mod_test_target.append(1)
            else:
                mod_test_target.append(0)
    
    return (mod_test_target, pred_list)

time: 7.01 ms


In [ ]:
for _class in class_names:
    
    actual_class = _class
    pred_class = 'pred_'+_class
    print ('Class Name : {}'.format(actual_class))
    logger.info('Class Name : {}'.format(actual_class))
    images, checking_label = make_train_triplets(_class, sample_train_data, 150)
    test_images, test_checking_label, false_inference_index = make_test_triplets(_class, sample_train_data)
    
    first_images, second_images, third_images, target = ref_train_image_triplets(images, checking_label)
    test_first_images, test_second_images, test_third_images, test_target = ref_test_image_triplets(test_images, test_checking_label)
    
    ref_train_data = TensorDataset(first_images, second_images, third_images, target)
    train_dataloader = DataLoader(ref_train_data, batch_size = args['batch_size'], shuffle = True, num_workers = 8, pin_memory = True)

    test_data = TensorDataset(test_first_images, test_second_images, test_third_images, test_target)
    test_dataloader = DataLoader(test_data, batch_size = args['batch_size'], shuffle = False, num_workers = 8, pin_memory = True)

    criterion = torch.nn.MarginRankingLoss(margin = args['loss_margin'])
    optimizer = optim.Adam (model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-5)
    
    mod_test_target, pred_list = train_and_eval(ref_model, train_dataloader, test_dataloader)
    
    # Calculating the ROC-AUC Scores before and after the few shot learning algorithm.
    sample_train_data_copy_pred_class = copy.deepcopy(sample_train_data[pred_class])
    i = 0
    for index in false_inference_index:
        sample_train_data_copy_pred_class[index] = pred_list[i]
        i += 1
        
    sample_train_data['FSL_pred_'+_class] = sample_train_data_copy_pred_class
    
    print ('Before FSL - ') 
    logger.info('Before FSL - ')
    ppv, npv = predictive_value(confusion_matrix(sample_train_data[actual_class], sample_train_data[pred_class]))
    print ('Positive Predictive Value : {}'.format(ppv))
    logger.info('Positive Predictive Value : {}'.format(ppv))
    print ('Negative Predictive Value : {}'.format(npv))
    logger.info('Negative Predictive Value : {}'.format(npv))
    
    print ('After FSL - ')
    logger.info('After FSL - ')
    ppv, npv = predictive_value(confusion_matrix(sample_train_data[actual_class], sample_train_data_copy_pred_class))
    print ('Positive Predictive Value : {}'.format(ppv))
    logger.info('Positive Predictive Value : {}'.format(ppv))
    print ('Negative Predictive Value : {}'.format(npv))
    logger.info('Negative Predictive Value : {}'.format(npv))
    print ('--------------------------------------')
    logger.info('--------------------------------------')

Class Name : No Finding
False Negatives - 3785 False Positives - 578
False Negatives - 3785 False Positives - 578


  0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.1993531882762909


 20%|██        | 1/5 [06:31<26:07, 391.85s/it]

Train loss: 0.09878809005022049


...

Train loss: 0.00091033976059407


 80%|████████  | 4/5 [09:01<02:15, 135.17s/it]

Train loss: 0.0003031252126675099


100%|██████████| 5/5 [11:19<00:00, 135.81s/it]


Before FSL - 
Positive Predictive Value : 100.0
Negative Predictive Value : 0.0
After FSL - 
Positive Predictive Value : 100.0
Negative Predictive Value : 40.0
--------------------------------------
Class Name : Cardiomegaly
False Negatives - 1671 False Positives - 922
False Negatives - 1671 False Positives - 922
